In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# 파이캐럿 사용
#!pip install pycaret
#!pip install Jinja2
#!pip install markupsafe==2.1.1
#!pip install xgboost
#!pip install catboost

In [2]:
#file_path = '/content/drive/MyDrive/Text_Mining/'
file_path = ''
file_name_01 = 'final_total_df_01_res_20.csv'
file_name_03 = 'final_total_df_03_res_22.csv'
file_name_06 = 'final_total_df_06_18.csv'

- 시현을 위한 배포이므로 
    1. 비교적 비용이 낮은 레스토랑
    2. 비교적 비용이 높은 레스토랑
    3. 보트 관련 업체
    4. 서적 관련 업체
    5. 호텔 
    
- 가게를 사용해 구현 -> 필요한 데이터셋 3개만 불러옴

In [3]:
import pandas as pd
import ast

total_df_01 = pd.read_csv(file_path + file_name_01)
total_df_03 = pd.read_csv(file_path + file_name_03)
total_df_06 = pd.read_csv(file_path + file_name_06)

/var/tmp/ipykernel_676/1830159039.py:4: DtypeWarning: Columns (17,25,27,30,32,33,35,36,37,39,41,45,51,52,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  total_df_01 = pd.read_csv(file_path + file_name_01)
/var/tmp/ipykernel_676/1830159039.py:5: DtypeWarning: Columns (25,33,36,43,58,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  total_df_03 = pd.read_csv(file_path + file_name_03)
/var/tmp/ipykernel_676/1830159039.py:6: DtypeWarning: Columns (25,28,29,32,34,36,44,46,47,50,64,65,66,67,68,69,70,71,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  total_df_06 = pd.read_csv(file_path + file_name_06)


## 준비 : dataset으로 샘플 만들기!

In [4]:
try:
    total_df_01 = total_df_01.drop('Unnamed: 0', axis=1)
    total_df_03 = total_df_03.drop('Unnamed: 0', axis=1)
    total_df_06 = total_df_06.drop('Unnamed: 0', axis=1)
    
except:
    print('No index column')

final_df_01 = total_df_01.drop('text', axis=1)
final_df_03 = total_df_03.drop('text', axis=1)
final_df_06 = total_df_06.drop('text', axis=1)

category = 'summary'

remove_col = ['review_id','user_id','business_id', category, 'date','name','is_open']
# remove_col = ['review_id','user_id','business_id','date','name','is_open']


final_df_01 = final_df_01.drop(remove_col, axis = 1)

final_df_01 = final_df_01.replace(False, 0)
final_df_01 = final_df_01.replace('False', 0)
final_df_01 = final_df_01.replace(True, 1)
final_df_01 = final_df_01.replace('True', 1)

# Identify object type columns
object_columns = final_df_01.select_dtypes(include='object').columns

# Convert object columns to numeric or categorical
for column in object_columns:
    try:
        final_df_01[column] = pd.to_numeric(final_df_01[column], errors='coerce')
    except ValueError:
        final_df_01[column] = final_df_01[column].astype('category')

In [5]:
final_df_03 = final_df_03.drop(remove_col, axis = 1)

final_df_03 = final_df_03.replace(False, 0)
final_df_03 = final_df_03.replace('False', 0)
final_df_03 = final_df_03.replace(True, 1)
final_df_03 = final_df_03.replace('True', 1)

# Identify object type columns
object_columns = final_df_03.select_dtypes(include='object').columns

# Convert object columns to numeric or categorical
for column in object_columns:
    try:
        final_df_03[column] = pd.to_numeric(final_df_03[column], errors='coerce')
    except ValueError:
        final_df_03[column] = final_df_03[column].astype('category')

In [6]:
final_df_06 = final_df_06.drop(remove_col, axis = 1)

final_df_06 = final_df_06.replace(False, 0)
final_df_06 = final_df_06.replace('False', 0)
final_df_06 = final_df_06.replace(True, 1)
final_df_06 = final_df_06.replace('True', 1)

# Identify object type columns
object_columns = final_df_06.select_dtypes(include='object').columns

# Convert object columns to numeric or categorical
for column in object_columns:
    try:
        final_df_06[column] = pd.to_numeric(final_df_06[column], errors='coerce')
    except ValueError:
        final_df_06[column] = final_df_06[column].astype('category')

- user 정보를 위한 임의의 data 추출

In [7]:
sample_data_01 = final_df_01.iloc[1566:1567]
sample_data_03 = final_df_03.iloc[1567:1568]
sample_data_06 = final_df_06.iloc[1566:1567]

# 1. summary

In [8]:
def summary(review):
    # Create a temporary DataFrame
    temp_df = pd.DataFrame([str(review)], columns=['text'])

    # Calculate word count
    temp_df['word_count'] = temp_df['text'].str.split().str.len()

    # Filter by word count
    temp_df = temp_df[temp_df['word_count'] <= 300]

    # If there is any text left after the filtering, convert the first one to string
    if not temp_df.empty:
        review_summary = temp_df['text'].iloc[0]
            
    else:
        print("입력 된게 없어요!!")

    return review_summary

# 2. text 전처리 & sentiment

In [9]:
#!pip install transformers

In [10]:
# 전처리에 필요한 라이브러리 import
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from transformers import pipeline
from pycaret.regression import *
from nltk.corpus import wordnet


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/visualinformaticslab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/visualinformaticslab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/visualinformaticslab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/visualinformaticslab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
class Splitter(object):
    """
    split the document into sentences and tokenize each sentence
    """
    def __init__(self):
        self.splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):
        """
        out : ['What', 'can', 'I', 'say', 'about', 'this', 'place', '.']
        """
        # split into single sentence
        sentences = self.splitter.tokenize(text)
        # tokenization in each sentences
        tokens = [self.tokenizer.tokenize(sent) for sent in sentences]
        return tokens

class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v)
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(word, lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]
        return pos_tokens

def lemmatize_text(text):
    # Initialize required classes and objects
    lemmatizer = WordNetLemmatizer()
    splitter = Splitter()
    lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

    # Step 1: Split document into sentence followed by tokenization
    tokens = splitter.split(text)

    # Step 2: Lemmatization using pos tagger
    lemma_pos_token = lemmatization_using_pos_tagger.pos_tag(tokens)

    # Create lemmatized review
    lemmatized_text = ' '.join(word[1] for words in lemma_pos_token for word in words)

    return lemmatized_text


In [12]:
# Preprocess the text data
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

custom_stopwords = ['(', ')']

#custom stopword 추가
stop_words.update(custom_stopwords)
stop_words

def tokenize_text(text):
    # Import NLTK stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))

    result = ''
    word_tokens = word_tokenize(text)
    word_tokens = pos_tag(word_tokens)

    for word,tag in word_tokens:

        # Convert tokens to lowercase
        word = word.lower()

        # Remove stop words
        if word not in stop_words:
            result += ' ' + word

    return result


In [13]:
def get_sentiment_score(text):
    sentiment_score = None
    # 경고문 emoji(이모티콘) 라이브러리 설치
    # 우리 텍스트문에 이모티콘 있나 탐색해보고 있으면 설치


    data = ["I love you", "I hate you"]

    # bertweet_base_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis", device=0)
    bertweet_base_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")


    try:
        sentiment = bertweet_base_model(text)[0]
        label = sentiment['label']
        score = sentiment['score']

        # Map sentiment_value based on the label
        if label == 'POS':
            sentiment_score = score
        elif label == 'NEG':
            sentiment_score = 1 - score
        else:
            sentiment_score = 0.5

    except Exception as e:
        print("An error occurred: ", str(e))

    return sentiment_score

- 밑에 코드 실행하면 됨!

In [14]:
def preprocess_and_sentiment(review_summary):
    temp_text = lemmatize_text(review_summary)
    temp_text = tokenize_text(temp_text)

    sentiment_score = get_sentiment_score(temp_text)

    return sentiment_score

# 3. pycaret 모델 불러오기!

- 일단 6번으로 가정하고! 진행하고 다 한다음에 여기 if로 해서 수정하면 됨!!
    - if categories 해서 ㅇㅇ

In [15]:
#def choice_model():

def pycaret_model(final_model,sample_data):

    prediction = predict_model(final_model, data=sample_data)

    user_mean = 3
    user_var = 0.7
    predict_zscore = prediction['prediction_label'].values[0]
    # 리뷰에 대해 star(specific_stars)를 입력했을 때 상대적 중요도 z-score 표현 식.
    z_score = (predict_zscore - user_mean) / np.sqrt(user_var)
    
    return z_score

# 4. 웹으로 띄웁시다!


In [16]:
import gradio as gr
import numpy as np

# 모델 함수 (여기에서는 예시로 단순한 로직을 사용하였습니다. 실제 모델 로직으로 변경해주세요)
def calculate_tip(store, price, review):
    # 입력: 리뷰 텍스트 (문자열), 음식 가격
    # 출력: 팁 (문자열)
    price = float(price)
    
    if len(review) <= 0:
        return "리뷰를 입력해주세요"
    
    # 데이터 summary
    summary_review = summary(review)
    # 데이터 전처리&sentiment
    handled_review = preprocess_and_sentiment(summary_review)    
    

    # 가게에 따라 
    if store == "4 Kids Books and Toys":
        
        return "팁을 주지 않으셔도 되지만, 좋은 경험이었다면 작은 성의를 표시하셔도 좋을 것 같습니다."
    
    elif store == "Tsevi's Pub And Grill":
        
        final_model_01 = load_model(file_path + 'model_final_total_df_01_res_20.csv')
        sample_data_01['sentiment_score'] = [handled_review]

        predict_zscore =  pycaret_model(final_model_01, sample_data_01)
        defalut = 0.2  # <- 얘도 제어문 처리!

    elif store == "Mo's a Place For Steaks":
        final_model_03 = load_model(file_path + 'model_final_total_df_03_res_22.csv')
        sample_data_03['sentiment_score'] = [handled_review]

        predict_zscore =  pycaret_model(final_model_03, sample_data_03)
        defalut = 0.22  # <- 얘도 제어문 처리!

    elif store == "Thriller Clearwater":
        final_model_06 = load_model(file_path + 'model_final_total_df_06_18.csv')
        sample_data_06['sentiment_score'] = [handled_review]

        predict_zscore =  pycaret_model(final_model_06, sample_data_06)
        defalut = 0.18  # <- 얘도 제어문 처리!

    elif store == "Spring Mount Hotel":
        
        return "Housekeeping에게 2~3dollar, Valet에게 3~5 doaller, Bellman에게 1~2 dollar 정도로 팁을 주시는 것을 추천합니다"
    
    else:
        return "가게를 선택하지 않았습니다."
    
    # 있다고 가정!

    tip_stdev = 0.01

    # 최종 팁 스코어 계산
    tip_percent = defalut + predict_zscore * tip_stdev
    
    output_price = price * tip_percent
    
    output_price = int(output_price)
    tip_percent = tip_percent*100
    
    tip_percent = "{:.2f}".format(tip_percent)
    
    output_tip_percent = ""
    
    return f"{tip_percent}%로 팁을 주시는 것을 추천합니다. \n\n계산 결과 : ${output_price}"


iface = gr.Interface(
    fn=calculate_tip, # 위에서 정의한 함수
    inputs=[
        gr.inputs.Dropdown(["4 Kids Books and Toys", "Tsevi's Pub And Grill", "Mo's a Place For Steaks", "Thriller Clearwater", "Spring Mount Hotel"], label="방문할 가게를 선택해주세요:"),
        gr.inputs.Number(label="구매 가격을 입력해주세요:"),
        gr.inputs.Textbox(lines=5, placeholder="이 가게에 대한 리뷰를 여기에 입력해주세요...")
    ],
    outputs=gr.outputs.Textbox(),
)

iface.launch(share=True)




Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://699702d059b54f2f73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Transformation Pipeline and Model Successfully Loaded


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Transformation Pipeline and Model Successfully Loaded


...

Transformation Pipeline and Model Successfully Loaded


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Transformation Pipeline and Model Successfully Loaded


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Transformation Pipeline and Model Successfully Loaded
